In [29]:
import glob
import json
import os
%run GameFunctions.ipynb
%run PrintFunctions.ipynb
%run Stats.ipynb

In [1]:
class Player:
    def __init__(self, name = '', net = None):
        if name == '':
            self.name = Names.getRandomName() + "_" + Names.getRandomName()
        else:
            self.name = name
        self.team = ProcessFunctions.getRandomTeam()
        self.wins = 0
        self.games = 0
        self.turns = 0
        if net is None:
            self.net = SequentialPkmnPlayer()
        else:
            self.net = net
        self.opt = tf.keras.optimizers.Adam(
            learning_rate=tf.Variable(0.0001),
            beta_1=tf.Variable(0.9),
            beta_2=tf.Variable(0.999),
            epsilon=tf.Variable(1e-7),
            decay = tf.Variable(0.0),
        )
        self.ckpt = tf.train.Checkpoint(self.net, optimizer = self.opt)
        self.manager = tf.train.CheckpointManager(self.ckpt, directory = os.getcwd() + "/players/" + self.name + "/checkpoints/", max_to_keep=5)
    def getJsonDict(self):
        pDict = {}
        pDict['name'] = self.name
        pDict['team'] = self.team
        pDict['wins'] = int(self.wins)
        pDict['games'] = int(self.games)
        pDict['turns'] = int(self.turns)
        return json.dumps(pDict)
    def writeJson(jsonStr, path):
        with open(path + "/playerJson.txt", "w") as player:
            player.truncate(0)
            player.write(jsonStr)
    def readJson(path):
        with open(path + "/playerJson.txt", "r") as player:
            jsonDict = json.loads(player.read())
        return jsonDict
    def save(self):
        path = os.getcwd() + "/players/" + self.name
        if not os.path.isdir(path):
            os.mkdir(path)
            formattedPrint(f"Created dir for player {self.name}", location = "PLAYERS")
        Player.writeJson(self.getJsonDict(), path)
        save_path = self.manager.save()
        formattedPrint(f"saved to player {save_path}", location = "PLAYERS")
    def load(self, name):
        path = os.getcwd() + "/players/" + name
        jsonDict = Player.readJson(path)
        self.name = jsonDict['name']
        self.wins = jsonDict['wins']
        self.games = jsonDict['games']
        self.turns = jsonDict['turns']
        _ = self.net(tf.zeros([1, SEQUENTIALINPUTSHAPE[0]], tf.float32))
        self.ckpt.restore(self.manager.latest_checkpoint)
        formattedPrint(f"loaded player {self.name}", location = "PLAYERS")
    def setRandomTeam(self):
        self.team = ProcessFunctions.getRandomTeam()
    def updateStats(self, turns, didWin):
        self.games += 1
        if didWin:
            self.wins += 1
        self.turns += turns
        Statistics.winList[self.name] = self.wins
        Statistics.gamesPlayed += 1
    def actor(self, inpt):
        return self.net(inpt)[0]
    def critic(self, inpt):
        return self.net(inpt)[1]

Thu Mar  3 17:10:19 2022, PLAYERS:: Created dir for player Taryn_Catya
Thu Mar  3 17:10:19 2022, PLAYERS:: Saved player Taryn_Catya
